###  Read the PDF

In [7]:
import fitz  # PyMuPDF
import pikepdf
import os
import tqdm

MARKET = "kospi"

## 디버깅 - 오류난 파일 체크

In [ ]:
import os

def list_unique_files(dir1, dir2):
    # Get the list of files in both directories
    files_dir1 = set(os.listdir(dir1))
    files_dir2 = set(os.listdir(dir2))

    # Find files that are in dir1 but not in dir2
    unique_to_dir1 = files_dir1 - files_dir2

    # Find files that are in dir2 but not in dir1
    unique_to_dir2 = files_dir2 - files_dir1

    # Convert sets to lists
    unique_to_dir1 = list(unique_to_dir1)
    unique_to_dir2 = list(unique_to_dir2)

    return unique_to_dir1+ unique_to_dir2

MARKET = "kospi"
for year in range(2000,2025):
    # Example usage
    dir1 = f'/Users/jiho/Projects/kubs/data/report/{MARKET}/{year}' # path to origninal pdf files
    dir2 = f'/Users/jiho/Projects/kubs/data/report/extracted/{MARKET}/{year}'

    unique_files = list_unique_files(dir1, dir2)

    print(f"{year}. {len(unique_files)}")
    print(f"{unique_files}, ")

## PDF 페이지 추출 및 압축 함수

In [8]:
# Function to search for a text in the PDF
import subprocess

def search_for_text(document, search_text, start_page=0):
    for page_num in range(start_page, document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        if search_text in text:
            return page_num
        
    return -1


def compress_pdf(input_path, output_path, image_quality=10):
    with pikepdf.open(input_path) as pdf:
        # Iterate over all the images in the PDF and compress them
        for page in pdf.pages:
            for image in page.images:
                with pdf.open_image(image) as img:
                    img.compress(pikepdf.ImageFormat.JPEG, quality=image_quality)
        pdf.save(output_path)



def compress_pdf_with_ghostscript(input_path, output_path):
    gs_command = [
        "gs",
        "-sDEVICE=pdfwrite",
        "-dCompatibilityLevel=1.4",
        "-dPDFSETTINGS=/screen",
        "-dNOPAUSE",
        "-dQUIET",
        "-dBATCH",
        f"-sOutputFile={output_path}",
        input_path
    ]
    subprocess.run(gs_command)


def extract_pages(pdf_path, output_path, start_tag, end_tag):
    # Open the original PDF
    document = fitz.open(pdf_path)

    # Print the number of pages
    #print(f"Number of pages: {document.page_count}")

    # Create a new PDF
    new_document = fitz.open()

    # Find the start of the relevant section, starting from page 3 (index 2)
    start_page = search_for_text(document, start_tag, start_page=5)

    if start_page == -1:
        print(f"Start tag '{start_tag}' not found.")
        return

    # Find the end of the relevant section, starting from the page after the start page
    end_page = search_for_text(document, end_tag, start_page=start_page + 1)
    if end_page == -1:
        print(f"End tag '{end_tag}' not found.")
        return

    # Add the specified pages to the new PDF
    for page_num in range(start_page, end_page + 1):
        new_document.insert_pdf(document, from_page=page_num, to_page=page_num)
        
    # Save the new PDF
    temp_output_path = output_path.replace('.pdf', '_temp.pdf')
    new_document.save(temp_output_path)
    print(f"Extracted pages {start_page} to {end_page}")

    # Optimize the PDF
    #compress_pdf(temp_output_path, output_path, image_quality=10)
    compress_pdf_with_ghostscript(temp_output_path, output_path)
    
    # Remove the temporary file
    os.remove(temp_output_path)


    print(f"Optimized PDF saved as {output_path}")

    
    if ((end_page-start_page)<1):
        os.remove(output_path)
        return -1
    else :
        return end_page-start_page


def get_tags_for_year(year):
    if year == 2024:
        start_tag = "VI. 이사회 등 회사의 기관에 관한 사항"
        end_tag = "VII. 주주에 관한 사항"
    elif 2021 <= year <= 2023:
        start_tag = "VI. 이사회 등 회사의 기관에 관한 사항"
        end_tag = "VII. 주주에 관한 사항"
    elif 2014 <= year <= 2020:
        start_tag = ". 이사회 등 회사의 기관 및 계열회사에 관한 사항"
        end_tag = ". 주주에 관한 사항"
    elif 2009 <= year <= 2013: ## 수정
        start_tag = ". 이사회 등 회사의 기관 및 계열회사에 관한 사항"#"VI. 이사회 등 회사의 기관 및 계열회사에 관한 사항"
        end_tag = ". 주주에 관한 사항"
    elif 2007 <= year <= 2008:
        start_tag = "V. 이사회 등 회사의 기관 및 계열회사에 관한 사항" 
        end_tag = "VI. 주주에 관한 사항"
    elif year == 2006: ## 수정
        start_tag = ". 이사회 등 회사의 기관 및 계열회사에 관한 사항"  ## todo : 띄어쓰기 없게 한 번 더 
        end_tag =  ". 주주에 관한 사항"
    elif 2000 <= year <= 2005:
        start_tag = ". 지배구조 및 관계회사등의 현황"
        end_tag = ". 주식에 관한 사항"
    else:
        raise ValueError(f"Year {year} is out of the predefined range")
    
    return start_tag, end_tag


## PDF Compression ver 1. 한글 너무 깨짐

In [ ]:
#@ PDF COmpression ver 1. 한글 너무 깨짐
import os
import aspose.words as aw

# Define the function to resize PDF files
def resize_pdf(input_path, output_path, image_quality=80):
    renderer = aw.pdf2word.fixedformats.PdfFixedRenderer()
    pdf_read_options = aw.pdf2word.fixedformats.PdfFixedOptions()
    pdf_read_options.image_format = aw.pdf2word.fixedformats.FixedImageFormat.JPEG
    pdf_read_options.jpeg_quality = 90

    with open(input_path, 'rb') as pdf_stream:
        pages_stream = renderer.save_pdf_as_images(pdf_stream, pdf_read_options)

    builder = aw.DocumentBuilder()
    for i in range(len(pages_stream)):
        max_page_dimension = 1584
        page_setup = builder.page_setup
        set_page_size(page_setup, max_page_dimension, max_page_dimension)

        page_image = builder.insert_image(pages_stream[i])

        set_page_size(page_setup, page_image.width, page_image.height)
        page_setup.top_margin = 0
        page_setup.left_margin = 0
        page_setup.bottom_margin = 0
        page_setup.right_margin = 0

        if i != len(pages_stream) - 1:
            builder.insert_break(aw.BreakType.SECTION_BREAK_NEW_PAGE)

    save_options = aw.saving.PdfSaveOptions()
    save_options.cache_background_graphics = True

    builder.document.save(output_path, save_options)
    

def set_page_size(page_setup, width, height):
    page_setup.page_width = width
    page_setup.page_height = height

# Define the function to process all PDF files in a directory
def process_directory(directory_path):
    for root, _, files in os.walk(directory_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                input_path = os.path.join(root, file)
                output_path = os.path.join(root, f"resized/{file}")
                resize_pdf(input_path, output_path)
                print(f"Saved to {output_path}")


for year in [2024]:
    # Directory path
    directory_path = f'/Users/jiho/Projects/kubs/data/report/extracted/kospi/{year}'

    # Run the process
    #process_directory(directory_path)

In [44]:
def compress_pdf(input_path, output_path, image_quality=10):
    with pikepdf.open(input_path) as pdf:
        for page in pdf.pages:
            page_obj = pikepdf.Page(page)
            for image_name, image_obj in page_obj.images.items():
                with pdf.open_image(image_obj) as img:
                    img.save(image_obj, format='JPEG', quality=image_quality)
        pdf.save(output_path)

input_path = "/Users/jiho/Projects/kubs/data/report/extracted/kospi/2024/001630_종근당홀딩스.pdf"
output_path =  "/Users/jiho/Projects/kubs/data/001630_종근당홀딩스.pdf"
compress_pdf(input_path,output_path )

## PDF Compression . ver2 추출 및 압축

In [9]:
import os 
import tqdm

MARKET = "kospi"

error_2006 = ['004980_성신양회.pdf', '002820_SUN&L.pdf', '070960_모나용평.pdf', '008870_금비.pdf', '000180_성창기업지주.pdf', '008040_사조동아원.pdf', '[정정]000270_기아.pdf', '008250_이건산업.pdf', '021820_세원정공.pdf', '002070_비비안.pdf', '033250_체시스.pdf', '[정정]016380_KG스틸.pdf', '001080_만호제강.pdf', '003060_에이프로젠바이오로직스.pdf', '[정정]000150_두산.pdf', '002840_미원상사.pdf', '003610_방림.pdf', '005390_신성통상.pdf', '[정정]025750_한솔홈데코.pdf', '005800_신영와코루.pdf', '[정정]021820_세원정공.pdf']
error_2005 = ['[정정]003090_대웅.pdf', '[정정]003060_에이프로젠바이오로직스.pdf', '[정정]008040_사조동아원.pdf', '[정정]006740_영풍제지.pdf', '[정정]004720_팜젠사이언스.pdf', '[정정]033250_체시스.pdf', '004980_성신양회.pdf', '[정정]008250_이건산업.pdf', '[정정]001440_대한전선.pdf', '[정정]078930_GS.pdf', '[정정]001250_GS글로벌.pdf', '[정정]005380_현대자동차.pdf']
error_2008 = ['[정정]049800_우진플라임.pdf', '[정정]023960_에쓰씨엔지니어링.pdf', '[정정]006490_인스코비.pdf', '[정정]016710_대성홀딩스.pdf', '[정정]004140_동방.pdf', '012600_청호ICT.pdf', '[정정]006060_화승인더스트리.pdf', '004980_성신양회.pdf', '[정정]004840_DRB동일.pdf', '[정정]007340_DN오토모티브.pdf', '[정정]005390_신성통상.pdf']
error_2024 = ['111770_영원무역.pdf', '011330_유니켐.pdf', '009970_영원무역홀딩스.pdf', '005110_한창.pdf', '010600_웰바이오텍.pdf', '007610_선도전기.pdf', '001470_삼부토건.pdf']

for year in [2006]:
    ##range(2018, 1999, -1):
    error_list=[]
    input_dir = f'/Users/jiho/Projects/kubs/data/report/{MARKET}/{year}' # path to origninal pdf files
    output_dir = f'/Users/jiho/Projects/kubs/data/report/extracted/{MARKET}/{year}' # path to save extracted pdf files

    ### Make sure Directory exists
    #try:
    #    os.makedirs(output_dir)
    #    print(f"Directory '{output_dir}' created successfully")
    #except FileExistsError:
    #    print(f"Directory '{output_dir}' already exists")
    #except Exception as e:
    #    print(f"Error creating directory '{output_dir}': {e}")
    
    ## Processing by year
    tag = get_tags_for_year(year)
    print(tag)
    print(f"Processing {year}..")


    #for filename in tqdm.tqdm(os.listdir(input_dir)):
    for filename in tqdm.tqdm(error_2006):
        try:
            print(f"Processing {filename}..")
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(input_dir, filename)
                output_path = os.path.join(output_dir, f"{filename}")
                extracted_pages = extract_pages(pdf_path, output_path, tag[0], tag[1])
                if extracted_pages==-1:
                    error_list.append(filename)   
        
        except : 
            print(f"Error Processing {filename} ")
            error_list.append(filename)

    ## Print Error 
    error_file_path = f"/Users/jiho/Projects/kubs/progress/error/{MARKET}_extract_error_{year}.txt"
    with open(error_file_path, 'w') as f:
        for error in error_list:
            f.write(f"{error}\n")
    print(f"Error list saved to {error_file_path}")


Optimized PDF saved as /Users/jiho/Projects/kubs/data/report/extracted/kospi/2014/013890_지누스.pdf
Processing 013360_일성건설.pdf..
Extracted pages 47 to 55


 91%|█████████ | 856/945 [12:53<00:51,  1.72it/s]

Optimized PDF saved as /Users/jiho/Projects/kubs/data/report/extracted/kospi/2014/013360_일성건설.pdf
Processing 020000_한섬.pdf..
Extracted pages 99 to 108


 91%|█████████ | 857/945 [12:54<00:52,  1.67it/s]

Optimized PDF saved as /Users/jiho/Projects/kubs/data/report/extracted/kospi/2014/020000_한섬.pdf
Processing 005440_현대지에프홀딩스.pdf..
Extracted pages 51 to 57


...

Optimized PDF saved as /Users/jiho/Projects/kubs/data/report/extracted/kospi/2014/002990_금호건설.pdf
Processing [정정]005360_모나미.pdf..
Extracted pages 95 to 102


100%|█████████▉| 944/945 [14:21<00:01,  1.11s/it]

Optimized PDF saved as /Users/jiho/Projects/kubs/data/report/extracted/kospi/2014/[정정]005360_모나미.pdf
Processing 019180_티에이치엔.pdf..
Extracted pages 61 to 65


100%|██████████| 945/945 [14:21<00:00,  1.10it/s]

Optimized PDF saved as /Users/jiho/Projects/kubs/data/report/extracted/kospi/2014/019180_티에이치엔.pdf
Error list saved to /Users/jiho/Projects/kubs/progress/error/kospi_extract_error_2014.txt


## D4

In [1]:
import fitz  # PyMuPDF

def count_words_in_section(pdf_path, start_tag, end_tag):
    # Open the PDF file
    document = fitz.open(pdf_path)

    # Initialize variables
    start_page_num = -1
    end_page_num = -1
    start_pos = -1
    end_pos = -1
    word_count = 0


    # Find the start and end pages and positions of the section
    for page_num in range(2, document.page_count):
        page = document.load_page(page_num)
        text = page.get_text("text")
        if start_tag in text and start_page_num == -1:
            start_page_num = page_num
            start_pos = text.find(start_tag)
        if end_tag in text and start_page_num != -1:
            end_page_num = page_num
            end_pos = text.find(end_tag)
            break

    #print("page_num", start_page_num, end_page_num)

    # Extract the section text, excluding tables
    section_text = ""
    if start_page_num != -1 and end_page_num != -1:
        if start_page_num == end_page_num:
            section_text = extract_text_excluding_tables(document.load_page(start_page_num), start_pos, end_pos)
        else:
            section_text += extract_text_excluding_tables(document.load_page(start_page_num), start_pos)
            for page_num in range(start_page_num + 1, end_page_num):
                page = document.load_page(page_num)
                section_text += extract_text_excluding_tables(page)
            end_page = document.load_page(end_page_num)
            section_text += extract_text_excluding_tables(end_page, 0, end_pos)

    #print(section_text)
    # Count words in the section text
    word_count = len(section_text.split())
    
    return (start_page_num, end_page_num, word_count)

def extract_text_excluding_tables(page, start_pos=0, end_pos=None):
    text_blocks = page.get_text("blocks")
    text = ""
    for block in text_blocks:
        if block[6] == 0:  # block[6] is the block type; 0 is text, 1 is image, 2 is vector, etc.
            block_text = block[4]
            if end_pos is not None:
                block_text = block_text[start_pos:end_pos]
            text += block_text
    return text


def extract_info_from_filename(filename):
    parts = filename.split('_')
    stock_code = parts[0]
    corp_name = parts[1].replace('.pdf', '')
    return stock_code, corp_name

def get_tags_for_year(year):
    if year == 2024:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    elif 2021 <= year <= 2023:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    elif 2014 <= year <= 2020:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    elif 2009 <= year <= 2013:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    elif 2007 <= year <= 2008:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    elif year == 2006:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    elif 2000 <= year <= 2005:
        start_tag = 'II. 사업의 내용'
        end_tag = 'III. 재무에 관한 사항'
    else:
        raise ValueError(f"Year {year} is out of the predefined range")
    
    return (start_tag, end_tag)


In [ ]:
import pandas as pd
import tqdm
import os

MARKET = "kospi"

# Initialize an empty DataFrame
df = pd.DataFrame(columns=['stock_code', 'corp_name', 'year','word_count', 'filename', "start_page", "end_page"])


for year in range(2000,2024+1):
    rows=[]
    tag = get_tags_for_year(year)  
    
    error_list = []
    input_dir = f'/Users/jiho/Projects/kubs/data/report/{MARKET}/{year}'
    
    for filename in os.listdir(input_dir):
        try:
            print(f"Processing {filename}..")
            if filename.endswith(".pdf"):
                pdf_path = os.path.join(input_dir, filename)
                temp_data = count_words_in_section(pdf_path, tag[0], tag[1])
                #print(temp_data)

                stock_code, corp_name = extract_info_from_filename(filename)
                rows += [
                {
                    
                    'stock_code': stock_code,
                    'corp_name': corp_name,
                    'year': year,
                    'word_count': temp_data[2],
                    'filename': filename,
                    "start_page": temp_data[0], 
                    "end_page" : temp_data[1],
                    
                }
            ]
    
        except Exception as e:
            print(f"Error processing {filename}: {e}")
            error_list.append(filename)

    
    error_file_path = f"/Users/jiho/Projects/kubs/progress/error/[D4]{MARKET}_error_{year}.txt"
    with open(error_file_path, 'w') as f:
        for error in error_list:
            f.write(f"{error}\n")


    df = pd.DataFrame(rows)
    # Save the DataFrame to a CSV file
    df.to_csv(f"/Users/jiho/Projects/kubs/result/[D4]{MARKET}_{year}.csv", index=False)
    print("Data saved to word_counts.csv")

In [1]:
import pandas as pd

# 엑셀 파일 불러오기
file_path_d4 = '/Users/jiho/Projects/kubs/result/D4.사업의내용단어수/[D4]kospi_2024.csv'
file_path = "/Users/jiho/Projects/kubs/result/[Result] 2021-2024 - 2024.csv"
df1 = pd.read_csv(file_path_d4)
df2 = pd.read_csv(file_path)

df1

,stock_code,corp_name,year,word_count,filename,start_page,end_page
0,001630,종근당홀딩스,2024,25843,001630_종근당홀딩스.pdf,23,137
1,[정정]007340,DN오토모티브,2024,3822,[정정]007340_DN오토모티브.pdf,7,24
2,030720,동원수산,2024,5590,030720_동원수산.pdf,5,28
3,021050,서원,2024,3475,021050_서원.pdf,14,33
4,003780,진양산업,2024,1965,003780_진양산업.pdf,10,22
...,...,...,...,...,...,...,...
1013,460860,동국제강,2024,4402,460860_동국제강.pdf,5,33
1014,100250,진양홀딩스,2024,5254,100250_진양홀딩스.pdf,11,40
1015,002310,아세아제지,2024,6460,002310_아세아제지.pdf,15,39
1016,002990,금호건설,2024,4509,002990_금호건설.pdf,21,40


In [3]:
df1 = df1[["corp_name","word_count","start_page","end_page" ]]
df1.rename(columns={"corp_name":"기업명","word_count":"D4\n(사업의 내용 단어 수)" }, inplace=True)
df1

/var/folders/ls/71tm5nzj7m1dbp2xpq6gbkcc0000gn/T/ipykernel_49096/4276639696.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1.rename(columns={"corp_name":"기업명","word_count":"D4 (사업의 내용 단어 수)" }, inplace=True)


,기업명,D4 (사업의 내용 단어 수),start_page,end_page
0,종근당홀딩스,25843,23,137
1,DN오토모티브,3822,7,24
2,동원수산,5590,5,28
3,서원,3475,14,33
4,진양산업,1965,10,22
...,...,...,...,...
1013,동국제강,4402,5,33
1014,진양홀딩스,5254,11,40
1015,아세아제지,6460,15,39
1016,금호건설,4509,21,40


In [7]:
merged_df = pd.merge( df2,df1, on='기업명', how='inner')
merged_df.to_csv("/Users/jiho/Projects/kubs/result/2024_test_result.csv")